In [ ]:
# import the libraries
import cv2
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import PIL
import random
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
# initialize the constants: 
# Initial Learning Rate
# Epochs
# Batch Size
# Length 
# Width 
INITIAL_LEARINGRATE = .001
EPOCHS = 10
BATCH_SIZE = 32
LENGTH = 224
WIDTH = 224

Exploratory Data Analysis

In [ ]:
# images without covid
wOut_Covid = list(pathlib.Path("../DATA").glob('0/*'))
# images with covid
w_Covid = list(pathlib.Path("../DATA").glob('1/*'))


In [ ]:
print('There are', len(wOut_Covid), 'images without Covid')
print('There are', len(w_Covid), 'images without Covid')

plt.bar(['Without Covid', 'With Covid'], [len(wOut_Covid), len(w_Covid)], align = 'center')
plt.title('Number of images with and without Covid')
plt.show()

In [ ]:
# grab the image path for the images without covid
image_dir = "../DATA/0/"

# Create a 4x4 subplot
fig, axs = plt.subplots(4, 4, figsize=(16, 16))
axs = axs.ravel() # flatten the axes array

# Get a list of all image files in the directory
image_files = [filename for filename in os.listdir(image_dir)]

# Randomly select 16 images from the list
selected_files = random.sample(image_files, k=16)

# Loop through the selected files
for i, filename in enumerate(selected_files):
    # Load the image using matplotlib
    img = plt.imread(os.path.join(image_dir, filename))

    # Display the image in the appropriate subplot
    axs[i].imshow(img.astype("uint8"))
    axs[i].axis('off')
     
# show the plot
plt.suptitle('Dataset of Images Without Covid')
plt.tight_layout()
plt.show() 

In [ ]:
# grab the image path for the images with covid
image_dir = "../DATA/1/"

# Create a 4x4 subplot
fig, axs = plt.subplots(4, 4, figsize=(16, 16))
axs = axs.ravel() # flatten the axes array

# Get a list of all image files in the directory
image_files = [filename for filename in os.listdir(image_dir)]

# Randomly select 16 images from the list
selected_files = random.sample(image_files, k=16)

# Loop through the selected files
for i, filename in enumerate(selected_files):
    # Load the image using matplotlib
    img = plt.imread(os.path.join(image_dir, filename))

    # Display the image in the appropriate subplot
    axs[i].imshow(img.astype("uint8"))
    axs[i].axis('off')
     
# show the plot
plt.suptitle('Dataset of Images With Covid')
plt.tight_layout()
plt.show() 

Loading the Preprocessing the Data

In [ ]:
# take the images and put them into lists
image_path = list(paths.list_images(r"..\DATA"))

In [ ]:
# create empty list for the data and the labels
data = list()
labels = list()

In [ ]:
# now we want to loop over all the images in the files and then rescale them to a 224x224 aspect ratio. This will be done via the CV library
for x in image_path:
    # getting the classlabel
    label = x.split(os.path.sep)[-2]

    # now, we load the image, change the color to RGB and then resize it
    image = cv2.imread(x)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = cv2.resize(image,(LENGTH,WIDTH))

    # update our label and data list
    labels.append(label)
    data.append(image)


In [ ]:
# convert the data and labels to Numpy Arrays while scaling the pixel intensity
data = np.array(data) / 255.0 # 255 RGB colors
data.reshape(-1,1)
labels = np.array(labels)

In [ ]:
# one-hot encoding the models
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
# creating our train and test data. Using standard practice, 80% of the dataset will be used for training 
# while the 20% will be used for testing
trainX, testX, trainY, testY = train_test_split(data,labels,test_size=.2, stratify=labels,random_state=42)

In [ ]:
# initialize the training data augmentation object
train_augment = ImageDataGenerator(
    rotation_range =15,
    fill_mode = 'nearest'
)

Creating the Model using VGG16 Base Model

In [ ]:
# MODEL IS HERE!

Compiling the Model

In [ ]:
# using the ADAM optimizer to optimize
optimizer = Adam(learning_rate=INITIAL_LEARINGRATE, weight_decay = INITIAL_LEARINGRATE/EPOCHS)

In [ ]:
# compile the model
model.compile(
    optimizer = optimizer,
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

Training the Network

In [ ]:
# train the head of the network
head = model.fit(
    train_augment.flow(trainX,trainY,batch_size=BATCH_SIZE),
    steps_per_epoch = len(trainX)//BATCH_SIZE,
    validation_data = (testX,testY),
    validation_steps = len(testX) // BATCH_SIZE,
    epochs = EPOCHS
)

Making Predictions

In [ ]:
# predicting using the trained model
predict = model.predict(testX,batch_size=BATCH_SIZE)

In [ ]:
# for each image, find the index of the label corresponding with the largest predicted probability
predict = np.argmax(predict,axis=1)

In [ ]:
# now represent the prediction in a nicely formatted classification report
print(classification_report(testY.argmax(axis=1),predict,target_names=lb.classes_))

Confusion Matrix For Statistical Evaluation

In [ ]:
# create a confusion matrix consisting of accuracy, sensitivity, and specificity. 
confusionMatrix =confusion_matrix(testY.argmax(axis=1),predict)
total = sum(sum(confusionMatrix))

# caculate accuracy, sensitivity, and specificity
accuracy = (confusionMatrix[0,0] + confusionMatrix[1,1]) / total
sensitivity = confusionMatrix[0,0] / (confusionMatrix[0,0] + confusionMatrix[0,1])
specificity = confusionMatrix[1,1] / (confusionMatrix[1,0] + confusionMatrix[1,1])

In [ ]:
# display the output
print(confusionMatrix)
print("Accuracy:    ", round(accuracy,3))
print("Sensitivity: ", round(sensitivity,3))
print("Specificity: ", round(specificity,3))


Plot for Visual Representation

In [ ]:
# graph for visual representation
plt.plot(np.arange(0, EPOCHS), head.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), head.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), head.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), head.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on COVID-19 Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.show()